In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

# Learning Pandas

* Pandas provides a simpler API for array based data processing with support for non-numerical datatypes and joins, group-by, pivot ops that are particularly useful for Table-like data or time-series.

* Underlying storage is blocked and separated by data type instead of the contiguous storage of structured arrays in numpy). This allows faster processing in many cases.
* Simpler interfaces to common operations (file-loading, plotting, selection, and joining / aligning data) make it easy to do a lot of work in little code.
* Index arrays which mean that operations are always aligned instead of having to keep track of alignment yourself.
* Split-Apply-Combine is a powerful way of thinking about and implementing data-processing

## Containers

Pandas provides two main container types, ``Series`` & ``DataFrame``.

* A Series is a associative array of values. Each entry can be accessed by a label in addition to positional index.

* A DataFrame is 2D container with labels attached both dimensions (rows and columns). Both sets of labels are defined as an Index.

For some common operations on Series & DataFrames, we can review <a href="https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html">10 minutes to Pandas</a>.

### Explicit Indexing
Use .loc or iloc explicitly to indicate if you're using label based indexing or position based indexing, and if you're searching for matches in index or column dimension. Using the "[" operator can lead to unexpected results because in that case, pandas searches in both index and columns for matching labels.

### Selections for Duplicate Labels

In [2]:
df = pd.DataFrame([[100], [200], [300]], index=['a','a', 'b'], columns=['C1'])
df

,C1
a,100
a,200
b,300


In [3]:
# Returns a series
df.loc['b', :]

C1    300
Name: b, dtype: int64

In [4]:
# Returns dataframe

df.loc['a', :]

,C1
a,100
a,200


In [5]:
# Explicitly specify a list of index values to match
df.loc[['b'], :]

,C1
b,300


### map, apply, applymap and transform

map is defined only on Series and is used to map the values of the series to some other values. The argument is either a Mapping or a function.

In [6]:
# map cards to their corresponding points in blackjack
cards = pd.Series(data=['king', 'queen', 'ace', 'two'], name='cards')
pm = {'king': 10, 'queen': 10, 'ace': 1, 'two': 2}
cards.map(pm)

0    10
1    10
2     1
3     2
Name: cards, dtype: int64

applymap is the DataFrame equivalent of map. It is always applied per cell.

In [7]:
# Square every cell
cards = pd.DataFrame(data=[[1, 2], [3, 4]])
cards.applymap(np.square)

,0,1
0,1,4
1,9,16


transform is defined on both Series and DataFrame.

For a Series, the argument can be either a function or a list of functions. Must return a series of the same length.

For a DataFrame, transform applies the argument function(s) to row Series (axis=1) or each column Series (axis=0).

In [8]:
counts = pd.Series([1, 2, 3, 4])
counts.transform([np.square, lambda v: v+1])

,square,<lambda>
0,1,2
1,4,3
2,9,4
3,16,5


In [9]:
counts_df = pd.DataFrame([[1, 3, 7, 10], [5, 4, 7, 8]], columns=list('ABCD'))
counts_df.transform(np.square)

,A,B,C,D
0,1,9,49,100
1,25,16,49,64


In [10]:
# For each row, subtract the row max from each cell in the row
counts_df.transform(lambda row: row - row.max(), axis=1)

,A,B,C,D
0,-9,-7,-3,0
1,-3,-4,-1,0


apply is defined for both Series and DataFrame. It is the most flexible method to create a DataFrame or Series from another DataFrame or Series. This method can also accept a single function or a list of functions. The result can be an aggregate or the same size as input, but not both.

In [11]:
# Aggregated
s = pd.Series(np.arange(10))
s.apply([np.median, np.mean])

median    4.5
mean      4.5
dtype: float64

In [12]:
# Same size
s = pd.Series(np.arange(10))
s.apply(np.square)

0     0
1     1
2     4
3     9
4    16
5    25
6    36
7    49
8    64
9    81
dtype: int64

### Objects as Index

Using custom objects as Index is an easy way to pass extra information around.

In [13]:
from dataclasses import dataclass


@dataclass(frozen=True)
class User:
    id: int
    name: str
        
        
page_visits = pd.DataFrame(
    data=[
        [100, 150, 300],
        [130, 120, 115],
    ],
    columns=['jan', 'feb', 'march'],
    index=[User(1, 'A'), User(2, 'B')]
)

In [14]:
page_visits

,jan,feb,march
"User(id=1, name='A')",100,150,300
"User(id=2, name='B')",130,120,115


In [15]:
# Frozen dataclasses have a well-defined __hash__() implementation
page_visits.loc[User(1, 'A'), 'jan']

100

### Chain Function Calls

Instead of nesting function calls for functions that take and return DataFrames, like ``f1(f2(f3(df)))``, we can chain together calls using the ``DataFrame.pipe`` method.

In [16]:
# Calculate (1 + element)^2 for each element
def add_one(df):
    return 1 + df

s = pd.Series(np.arange(5))
s.pipe(add_one).pipe(np.square)

0     1
1     4
2     9
3    16
4    25
dtype: int64

## Reading & Writing Data

Pandas supports a read_X and a to_X to different storage types:

* csv
* pickle
* json
* sql (using a dbapi or sqlalchemy connection)
* parquet (requires pyarrow)
* xml
* bigquery
* clipboard

The <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html#pandas.read_csv">csv option</a> is probably most flexible.

* It's possible to read directly from http, ftp, s3, gs, and local filesystem
* Compressed files can be read directly
* For large files, we can read files line by line or chunk by chunk
* It's possible to read only a subset of columns

## Combining DataFrames

There are 3 different styles of combining DataFrames:

* ``concat`` Combines Series & DataFrames row-wise or column wise. 
* ``join`` Database style join along index of one DataFrame and index or column of another DataFrame.
* ``merge`` Database style join along index or columns of one DataFrame and index or columns of another DataFrame. This is the most flexible option.

### concat

In [17]:
## concat series into dataframe

s1 = pd.Series(data=[100, 200], name='A')
s2 = pd.Series(data=[1, 2], name='B')

pd.concat([s1, s2], axis=1)

,A,B
0,100,1
1,200,2


In [18]:
# Same, but use different names
pd.concat([s1, s2], axis=1, keys=['A_new', 'B_new'])

,A_new,B_new
0,100,1
1,200,2


In [19]:
# Can also use a dict
pd.concat({'A_new': s1, 'B_new': s2}, axis=1)

,A_new,B_new
0,100,1
1,200,2


In [20]:
# index-aligned by default

s1 = pd.Series(data=[100, 200], name='A')
s2 = pd.Series(data=[10, 20], name='B', index=[3, 4])

pd.concat([s1, s2], axis=1)

,A,B
0,100.0,NaN
1,200.0,NaN
3,NaN,10.0
4,NaN,20.0


In [21]:
# Use join=inner to only keep common indexes
# It's not possible out of the box to paste into a dataframe with 2 rows, we need to 
# reset_index(drop=True) for that
s1 = pd.Series(data=[100, 200], name='A', index=[1, 2])
s2 = pd.Series(data=[10, 20], name='B', index=[2, 3])

pd.concat([s1, s2], axis=1, join='inner')

,A,B
2,200,10


In [22]:
# Concatenating along rows - indexes are appended as is
s1 = pd.Series(data=[100, 200], name='A')
s2 = pd.Series(data=[10, 20], name='B')
pd.concat([s1, s2], axis=0)

0    100
1    200
0     10
1     20
dtype: int64

In [23]:
# Use ignore_index=True to make a new, unique index
pd.concat([s1, s2], axis=0, ignore_index=True)

0    100
1    200
2     10
3     20
dtype: int64

In [24]:
# For dataframes, concat works the same way, except it is both index aligned and column aligned
d1 = pd.DataFrame(data=[[1, 'A'], ['2', 'B']], columns=['id', 'name'])
d2 = pd.DataFrame(data=[[3, 'C'], ['4', 'D']], columns=['id', 'name'])

pd.concat([d1, d2], axis=0, ignore_index=True)

,id,name
0,1,A
1,2,B
2,3,C
3,4,D


In [25]:
# Here both name1 and name2 appear in the result with missing values nan padded
d1 = pd.DataFrame(data=[[1, 'A'], ['2', 'B']], columns=['id', 'name1'])
d2 = pd.DataFrame(data=[[3, 'C'], ['4', 'D']], columns=['id', 'name2'])

pd.concat([d1, d2], axis=0, ignore_index=True)

,id,name1,name2
0,1,A,NaN
1,2,B,NaN
2,3,NaN,C
3,4,NaN,D


In [26]:
# join='inner' keeps only the shared column
pd.concat([d1, d2], axis=0, ignore_index=True, join='inner')

,id
0,1
1,2
2,3
3,4


### Join

In [27]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [28]:
right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)
right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [29]:
# inner join left & right based on index
left.join(right, how='inner')

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


In [30]:
# When left & right have common column names, they are repeated
# In this case, we must provide an lsuffix and an rsuffix

left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

right = pd.DataFrame(
    {"A": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)
left.join(right, how='inner', lsuffix='_x', rsuffix='_y')

,A_x,B,A_y,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


### Merge

In [31]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)
left

,key1,key2,A,B
0,K0,K0,A0,B0
1,K0,K1,A1,B1
2,K1,K0,A2,B2
3,K2,K1,A3,B3


In [32]:
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)
right

,key1,key2,C,D
0,K0,K0,C0,D0
1,K1,K0,C1,D1
2,K1,K0,C2,D2
3,K2,K0,C3,D3


In [33]:
# Indexes are always discarded
left.merge(right, on=["key1", "key2"], how='inner')

,key1,key2,A,B,C,D
0,K0,K0,A0,B0,C0,D0
1,K1,K0,A2,B2,C1,D1
2,K1,K0,A2,B2,C2,D2


In [34]:
# we can check for duplicate keys using the validate argument, e.g. validate="one_to_one"
# This will fail because (K1, K0) appears multiple times in left
left.merge(right, on=["key1", "key2"], how='inner',  validate="one_to_one")

MergeError: Merge keys are not unique in right dataset; not a one-to-one merge

In [ ]:
# If the column names to be used for merge keys are different, we can specify them with left_on and right_on
# Non-key columns with the same name are auto-suffixed
left = pd.DataFrame(data=[[1, 'A'], [2, 'B']], columns=['id1', 'C'])
right = pd.DataFrame(data=[[1, 'C'], [3, 'D']], columns=['id2', 'C'])

left.merge(right, left_on='id1', right_on='id2')

## Aggregate

The main aggregate function in pandas is ``DataFrame.groupby()``. This is often called split-apply-combine, because the entire operation is split in 3 stages

* split stage splits the source DataFrame into multiple groups based on selected column values
* apply performs some operation on each group
* combine assembles a DataFrame out of the apply stage results.

All 3 stages can be customized.

In [35]:
# Read the list of historical members of US congress
data_url = 'https://theunitedstates.io/congress-legislators/legislators-historical.csv'

usecols = [
    'last_name', 
    'first_name',          
    'middle_name',       
    'birthday',
    'gender',                 
    'type',                 
    'state',                  
    'district',                        
    'party',
]
congress_df = pd.read_csv(data_url, usecols=usecols, parse_dates=['birthday'])
congress_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12049 entries, 0 to 12048
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   last_name    12049 non-null  object        
 1   first_name   12049 non-null  object        
 2   middle_name  8389 non-null   object        
 3   birthday     11499 non-null  datetime64[ns]
 4   gender       12049 non-null  object        
 5   type         12049 non-null  object        
 6   state        12049 non-null  object        
 7   district     10209 non-null  float64       
 8   party        11817 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 847.3+ KB


In [36]:
# What are all the unique parties
congress_df.party.unique()

array(['Anti-Administration', nan, 'Pro-Administration', 'Republican',
       'Federalist', 'Democratic Republican', 'Unknown', 'Adams',
       'Jackson', 'Jackson Republican', 'Crawford Republican', 'Whig',
       'Anti-Jacksonian', 'Adams Democrat', 'Nullifier', 'Anti Masonic',
       'Anti Jacksonian', 'Jacksonian', 'Democrat', 'Anti Jackson',
       'Union Democrat', 'Conservative', 'Ind. Democrat', 'Law and Order',
       'American', 'Liberty', 'Free Soil', 'Independent',
       'Ind. Republican-Democrat', 'Ind. Whig', 'Unionist',
       'States Rights', 'Anti-Lecompton Democrat',
       'Constitutional Unionist', 'Independent Democrat',
       'Unconditional Unionist', 'Conservative Republican',
       'Ind. Republican', 'Liberal Republican', 'National Greenbacker',
       'Readjuster Democrat', 'Readjuster', 'Union', 'Union Labor',
       'Populist', 'Silver Republican', 'Free Silver',
       'Democratic and Union Labor', 'Progressive Republican',
       'Progressive', 'Prohibit

In [37]:
congress_df[congress_df.party.isnull()]

,last_name,first_name,middle_name,birthday,gender,type,state,district,party
1,Bland,Theodorick,NaN,1742-03-21,M,rep,VA,9.0,NaN
2,Burke,Aedanus,NaN,1743-06-16,M,rep,SC,2.0,NaN
3,Carroll,Daniel,NaN,1730-07-22,M,rep,MD,6.0,NaN
4,Clymer,George,NaN,1739-03-16,M,rep,PA,-1.0,NaN
5,Contee,Benjamin,NaN,NaT,M,rep,MD,3.0,NaN
...,...,...,...,...,...,...,...,...,...
8412,Delgado,Francisco,Afan,1886-01-25,M,rep,PI,0.0,NaN
8432,Guevara,Pedro,NaN,1879-02-23,M,rep,PI,0.0,NaN
8591,Paredes,Quintin,NaN,1884-09-09,M,rep,PI,0.0,NaN
8879,Elizalde,Joaquin,Miguel,1896-08-02,M,rep,PI,0.0,NaN


In [38]:
# unique genders
congress_df.gender.unique()

array(['M', 'F'], dtype=object)

In [40]:
# How many members of each gender do we have?
congress_df.groupby('gender')['first_name'].count().to_frame(name='total_by_gender')

,total_by_gender
gender,
F,248
M,11801


In [46]:
# top 10 states by member count
count_by_state = congress_df.groupby('state')['first_name'].count().to_frame(name='total_by_state')
count_by_state = count_by_state.sort_values(by='total_by_state', ascending=False)
count_by_state.head(10)

,total_by_state
state,
NY,1467
PA,1053
OH,676
IL,488
VA,433
MA,427
KY,373
CA,367
NJ,359


In [52]:
# Male and Female rep count for every state
counts = congress_df.groupby(['state', 'gender'])['first_name'].count()
counts = counts.to_frame(name='totals')
counts.head(5)

totals
state gender        
AK    M           16
AL    F            4
      M          205
AR    F            5
      M          112

In [54]:
# male & female reps for NY
counts.loc["NY", :]

,totals
gender,
F,20
M,1447


In [64]:
# Which state has the biggest difference of Male vs Female reps?
def gdiff(group_df):
    """Diff the number of male & female reps per state
    If the value is missing, default to 0    
    """
    counts = group_df.gender.value_counts()
    n_diff = counts.get('M', 0) - counts.get('F', 0)
    return n_diff

counts = congress_df.groupby(['state']).apply(gdiff)
counts = counts.sort_values(ascending=False).to_frame(name='gender_diff')
counts.head()

,gender_diff
state,
NY,1427
PA,1039
OH,658
IL,462
VA,425


In [65]:
counts.loc['NY', :]

gender_diff    1427
Name: NY, dtype: int64

## Window Functions

## Time Series Processing

## > 2 Dimensions

## Resources

* <a href="https://pandas.pydata.org/docs/user_guide/index.html">User Guide</a>
* <a href="https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html">Official Tutorial</a>
* <a href="https://github.com/jvns/pandas-cookbook">Cookbook</a>